# Raster Analysis using Open-Source Python

---

**A Tufts University Data Lab Workshop**\
Written by Uku-Kaspar Uustalu

Website: [tuftsdatalab.github.io/python-raster](https://tuftsdatalab.github.io/python-raster/)\
Contact: <datalab-support@elist.tufts.edu>\
Resources: [datalab.tufts.edu](https://sites.tufts.edu/datalab/)

Last updated: `2021-11-30`

---

This tutorial will walk you thorugh conducting a raster risk assesment using open-source Python libraries. Note that similar techiniques could also be used to conduct a raster suitability analysis instead. The workshop is largely based on [a similar raster risk assesment tutorial](https://sites.tufts.edu/gis/files/2013/11/RiskAssessment_MalariaInEthiopia_tutorial_10.6.1.pdf) written by Carolyn Talmadge for the Tufts University Data Lab. Unlike this notebook, the aformetioned tutorial utilizes ArcGIS Desktop (also known as ArcMap) to conduct the analysis. The following demonstrates how to implement a similar workflow using open-source Python libaries instead of traditional desktop GIS software.

## Introduction

Malaria is a serious acute illness transmitted via the bite of a mosquito. Symptoms include fever, headache, backache, joint pains, and vomiting. If not treated, malaria can quickly become life-threatening. According to the World Health Organization (WHO), roughly 75% of the land and 60% of the population of Ethiopia is exposed to malaria. The purpose of this analysis is to determine the overall risk of Malaria transmission throughout Ethiopia, considering the following factors:

- **Proximity to rivers** – Malaria is a water-related disease because it is transmitted via mosquitos who prefer to breed near rivers. Therefore, proximity to rivers results in higher risk of transmission.
- **Elevation** – Elevation is directly linked to climate, and mosquitos prefer warmer climates and lower elevations. Lower elevations increase mosquito population turnover, which means a higher risk of transmission.
- **Slope** – Gentle slopes equal higher susceptible areas for malaria incidence.
- **Land cover** – Mosquitos prefer damp areas, so land cover will affect the analysis.

## The Data

The `data` directory contains the following files:

- `elevation.tif` - digital elevation model (DEM) of Ethiopia and surrounding areas (GeoTIFF)
- `ethiopia.geojson` – the boundaries of Ethiopia (GeoJSON)
- `landcover.tif` - land cover raster for Ethiopia and surrounding areas (GeoTiFF)
- `rivers.geojson` – rivers of Ethiopia (GeoJSON)
- `zones.gojson` – boundaries of second-order administrative zones (GeoJSON)

To simplify the analysis, the `elevation.tif` and `landcover.tif` rasters have been clipped to the same extent with matching resolutions and projected to the [EPSG:32637 (UTM zone 37N)](https://epsg.io/32637) projected coordinate system using [`gdalwarp`](https://gdal.org/programs/gdalwarp.html). The coordinate system for the GeoJSON files is the [EPSG:4326 (WGS 84)](https://epsg.io/4326) unprojected coordinate system as per the GeoJSON standard.


## Analysis Overview

Here is a rough outline of the analysis we will perform:

1. Read in and visualize the raster datasets
2. **Land cover** > reclassify for land cover preferences
3. **Elevation (DEM)** > derive slope > reclassify for slope preferences (low slope = high risk)
4. **Elevation (DEM)** > reclassify for elevation preferences (low elevation = high risk)
5. **Rivers** > rasterize vector > get euclidean distance raster > reclassify (close = high risk) 
6. Calculate weighted and unweighted risk raster from all reclassified rasters
7. Mask risk rasters to the shape of Ethiopia
8. Use zonal statistics to determine average risk per zone


## Import Dependencies

The analysis will utilize the following libraries and modules.

In [ ]:
import rasterio
from rasterio.plot import show
from rasterio import features

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import richdem as rd
from scipy import ndimage
from rasterstats import zonal_stats

import contextily as cx

## Read Raster Datasets

The [`rasterio.open()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.html#rasterio.open) function can be used to open and read in raster datasets. The following will walk you throguh openting and reading in the elevation and landcover datasets using this function.

It is important to remember that just like the [`open()`](https://docs.python.org/3/library/functions.html#open) function in Python, the [`rasterio.open()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.html#rasterio.open) function establishes a connection to the file, which should later be closed. Hence, you should either use [`with`](https://docs.python.org/3/reference/compound_stmts.html#with) to open the file and extract all relevant information or make sure to use [`.close()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.close) to close the connection once you are done.

Antoher important thing to note is that the [`rasterio.open()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.html#rasterio.open) function returns a [`DatasetReader`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader) object, not the dataset itself. However, the [`DatasetReader`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader) object has numerous attributes and methods that allow you to read in various components of the raster dataset. For example, you can use the [`.meta`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.meta) attribute to access the metadata associated with the raster and you can use the [`.read()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.read) method to extract the raster data values as a matrix.

Both raster datasets should have the same coordinate system, extent, and resolution, but you should use the [`.meta`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.meta) attribute to compare the metadata for both files and ensure this is the case.

Once you have confirmed that the spatial attributes of both rasters are the same, use the [`.read()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.read) method with the appropriate band number to save the data matrices of the raster datasets into variables named `elevation` and `landcover`. Then pick one raster and use the appropriate attributes to extract all necessary spatial information into corresponding variables as well. Here is the information that is vital to the analysis:

- [`bounds`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.bounds) – the spatial extent of the rasters
- [`crs`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.crs) – the coordinate system used
- [`nodata`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.nodata) – the nodata value used (do this for each raster if the nodata values do not match)
- [`shape`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.shape) – number of rows and columns
- [`transform`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.transform) – how to translate from row/column indices to spatial coordinates
- [`res`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.res) – the raster resolution (dimensions of the raster cell)

Make sure to explore both the data matrices and the various metadata values once you have extracted them to ensure everything is as expected.

In [ ]:
# check if both files have the same metadata (spatial attributes)
with rasterio.open('./data/elevation.tif') as elev_file:
    with rasterio.open('./data/landcover.tif') as land_file:
        print(elev_file.meta == land_file.meta)

In [ ]:
# extract data matrices and raster attributes
with rasterio.open('./data/elevation.tif') as elev_file:
    with rasterio.open('./data/landcover.tif') as land_file:
        elevation = elev_file.read(1)
        landcover = land_file.read(1)
        bounds = elev_file.bounds
        crs = elev_file.crs
        nodata = elev_file.nodata
        shape = elev_file.shape
        transform = elev_file.transform
        res = elev_file.res
        meta = elev_file.meta

In [ ]:
meta

In [ ]:
res

In [ ]:
transform

In [ ]:
shape

In [ ]:
nodata

In [ ]:
crs

In [ ]:
bounds

In [ ]:
elevation

In [ ]:
landcover

## Visualize the Raster Datasets

Use the RasterIO [`show()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.plot.html#rasterio.plot.show) function to visualize both raster datasets and ensure they were read in correctly. Make sure to specify the `transform` to see actual spatial coordinates on the axes. Also use the `cmap` attribute to specify appropriate [Matplotlib colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html) for each dataset. Remember that land cover data is categorical, so you should use a categorical color map for that visualization.

In [ ]:
show(elevation, transform = transform, cmap = 'terrain')
plt.show()

In [ ]:
show(landcover, transform = transform, cmap = 'tab20')
plt.show()

## Reclassify Land Cover

Although the land cover raster contains numeric data, each number is representative of a land cover type. This information is usually stored in the raster attribute table accompanying the raster. The land cover raster we are using also has an accompanying raster attribute tablen availalbe, but unfortunately RasterIO is not yet able to read raster attribute tables. Hence, you have to use the table below to manually map the numerical cell values to their corresponding land cover types.

| Cell Value | Land Cover                       |
| :--------: | -------------------------------- |
|      0     | Water                            |
|      1     | Evergreen Needleleaf Forest      |
|      2     | Evergreen Broadleaf Forest       |
|      3     | Deciduous Needleleaf Forest      |
|      4     | Deciduous Broadleaf Forest       |
|      5     | Mixed Forests                    |
|      6     | Closed Shrublands                |
|      7     | Open Shrublands                  |
|      8     | Woody Savannas                   |
|      9     | Savannas                         |
|     10     | Grasslands                       |
|     11     | Permanent Wetlands               |
|     12     | Croplands                        |
|     13     | Urban and Built-Up Areas         |
|     14     | Cropland with Natural Vegetation |
|     15     | Snow and Ice                     |
|     16     | Barren or Sparsely Vegetated     |


We would like to assign a risk score ranging from one to five (with one denoting lowest risk and five denoting highest risk) to each of the land cover types as specified below.

| Land Cover Types                     | Land Cover Codes  | Risk Level     | Risk Score |
| ------------------------------------ | ----------------- | -------------- | :--------: |
| Water and Wetlands                   | 0, 11             | Very high risk |      5     |
| Croplands and Natural Vegetation     | 12, 14            | High risk      |      4     |
| Shrublands, Grasslands, and Savannas | 6, 7, 8, 9, 10    | Medium risk    |      3     |
| Forests and Urban/Built-Up Areas     | 1, 2, 3, 4, 5, 13 | Low risk       |      2     |
| Snow/Ice and Barren Land             | 15, 16            | Very low risk  |      1     |


Create a reclassified land cover raster as follows:
1. Use [`np.full()`](https://numpy.org/doc/stable/reference/generated/numpy.full.html) to create an empty array filled with `np.NaN` that matches the `shape` of the land cover raster.
2. Utilize [boolean indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html#boolean-array-indexing) to fill the newly created raster based on values of the `landcover` data matrix.

Once you have created the reclassified land cover raster, visualize it with using the appropriate `transform` and the `"RdYlBu_r"` (reversed red-yellow-blue) colormap.

In [ ]:
# create empty raster and populate based on reclassfied landcover values
land_reclass = np.full(shape, fill_value=np.NaN)
land_reclass[(landcover == 0) | (landcover == 11)] = 5
land_reclass[(landcover == 12) | (landcover == 14)] = 4
land_reclass[(landcover >= 6) & (landcover <= 10)] = 3
land_reclass[((landcover >= 1) & (landcover <= 5)) | (landcover == 13)] = 2
land_reclass[landcover >= 15] = 1
land_reclass

In [ ]:
show(land_reclass, transform=transform, cmap='RdYlBu_r')
plt.show()

## Calculate Slope Raster

Use [RichDEM](https://richdem.readthedocs.io/en/latest/) to calculate a slope raster as follows:
1. Use `rd.rdarray()` to create an `rdarray` version of the `elevation` matrix. Make sure to specify the appropriate `no_data` value!
2. Use `.to_gdal()` to convert the `transform` to the format expected by RichDEM and set that as the `.geotransform` attribute of the new `rdarray`.
3. Use [`rd.TerrainAttribute()`](https://richdem.readthedocs.io/en/latest/python_api.html#richdem.TerrainAttribute) to calculate the `"slope_percentage"` attribute.

Note that the values of the result matrix are slope percentages, meaning that they range from zero to one hundred. Make sure to take a look at the data matrix and visualize it using [`show()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.plot.html#rasterio.plot.show) to validate your work.

In [ ]:
# create an rdarray version of the elevation raster and set the transform
elevation_rd = rd.rdarray(elevation, no_data=nodata)
elevation_rd.geotransform = transform.to_gdal()
elevation_rd

In [ ]:
# calculate slope raster
slope = rd.TerrainAttribute(elevation_rd, attrib='slope_percentage')
slope

In [ ]:
# replace negative values with np.NaN
slope[slope < 0] = np.NaN
print(slope)

In [ ]:
show(slope, transform=transform, cmap='gray')
plt.show()

## Reclassify Slope Raster

Create a reclassified slope raster as follows:
1. Use [`np.full()`](https://numpy.org/doc/stable/reference/generated/numpy.full.html) to create an empty array filled with `np.NaN` that matches the `shape` of the slope raster.
2. Utilize [boolean indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html#boolean-array-indexing) to fill the newly created raster based on the following slope values.

| Min Slope (Inclusive) | Max Slope (Exclusive) |   Risk Level   | Risk Score |
| :-------------------: | :-------------------: | :------------: | :--------: |
|          ...          |           2%          | Very high risk |      5     |
|           2%          |           5%          |    High risk   |      4     |
|           5%          |          12%          |   Medium risk  |      3     |
|          12%          |          20%          |    Low risk    |      2     |
|          20%          |          ...          |  Very low risk |      1     |


Note that you will be repeating a similar workflow to reclassify other rasters numerous times. If you are up for the challenge, attempt to write a function for the reclassification procedure that will allow you to automate further reclassification attempts. Once you have created the reclassified slope raster, visualize it with using the appropriate `transform` and the `"RdYlBu_r"` (reversed red-yellow-blue) colormap.

In [ ]:
slope_reclass = np.full(shape, fill_value=np.NaN)
slope_reclass[slope < 2] = 5
slope_reclass[(slope >= 2) & (slope < 5)] = 4
slope_reclass[(slope >= 5) & (slope < 12)] = 3
slope_reclass[(slope >= 12) & (slope < 20)] = 2
slope_reclass[slope >= 20] = 1
slope_reclass

In [ ]:
show(slope_reclass, transform=transform, cmap='RdYlBu_r')
plt.show()

## Reclassify Elevation Raster

Create a reclassified elevation raster based on the following elevation values.

| Min Elevation (Inclusive) | Max Elevation (Exclusive) |   Risk Level   | Risk Score |
| :-----------------------: | :-----------------------: | :------------: | :--------: |
|            ...            |             0             | Very high risk |      5     |
|             0             |            500            |    High risk   |      4     |
|            500            |            1200           |   Medium risk  |      3     |
|            1200           |            3000           |    Low risk    |      2     |
|            3000           |            ...            |  Very low risk |      1     |


Once you have created the reclassified land elevation raster, visualize it with using the appropriate `transform` and the `"RdYlBu_r"` (reversed red-yellow-blue) colormap.

In [ ]:
elev_reclass = np.full(shape, fill_value=np.NaN)
elev_reclass[elevation < 0] = 5
elev_reclass[(elevation >= 0) & (elevation < 500)] = 4
elev_reclass[(elevation >= 500) & (elevation < 1200)] = 3
elev_reclass[(elevation >= 1200) & (elevation < 3000)] = 2
elev_reclass[elevation >= 3000] = 1
elev_reclass

In [ ]:
show(elev_reclass, transform=transform, cmap='RdYlBu_r')
plt.show()

## Read in and Rasterize Rivers Vector

We could like to consider distance to rivers in our analysis. To do so, we must first calculate an euclidean distance raster where the value of each cell is the distance to the nearest river from that cell. To do so, we must first read in and rasterize the rivers vector dataset as follows:

1. Use [`gpd.read_file()`](https://geopandas.org/docs/reference/api/geopandas.read_file.html#geopandas.read_file) to read in the rivers GeoJSON file.
2. Use the geopandas [`.to_crs()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html) method to reproject the vector data into the coordinate system used for the raster data.
2. Use [`features.rasterize()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html#rasterio.features.rasterize) from RasterIO to convert the rivers vector into a binary raster where zero denotes a river and one denotes the lack of a river.
    - Note that you should use the `"geometry"` column from the rivers GeoDataFrame as input.
    - Remember to specify the appropriate `transform`.
    - Use `default_value` to specify the value that should denote the presence of a river (0).
    - Use `fill` to specify the value that should denote the lack of a river (1).
    
Visualize both the shapefile and the raster to make sure the conversion worked as expected. If you have a hard time seeing the river pixels of the raster, use the `"flag"` colormap to bring them out more.

In [ ]:
# read in rivers shapefile
rivers = gpd.read_file('./data/rivers.geojson').to_crs(crs)
rivers.head()

In [ ]:
# plot rivers shapefle
rivers.plot()
plt.show()

In [ ]:
# convert rivers shapefile into a binary raster
rivers_raster = features.rasterize(rivers.geometry, out_shape=shape, fill=1,
                                   transform=transform, default_value=0)
rivers_raster

In [ ]:
show(rivers_raster, transform=transform, cmap='flag')
plt.show()

## Calculate Euclidean Distance Raster

Now we are ready to create an euclidean distance matrix that denotes distances to the nearest river. This is a relatively simple two-step process:
1. Use [`ndimage.distance_transform_edt()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.distance_transform_edt.html) from SciPy to calculate the euclidean distance transform for the binary rivers raster.
2. Multiply the distance transform raster with the raster cell size (width or length) to convert the distances from number of cells to spatial units.
    - The `res` attribute you saved earlier should contain the dimensions of the square raster cell.
    
Note that this workflow assumes the raster to have square cells, which is the case for us. Once completed, visualize the distance raster to validate your work.

In [ ]:
# confirm raster cells are square
res[0] == res[1]

In [ ]:
# calculate distance transform and multiply with cell size
rivers_distance = ndimage.distance_transform_edt(rivers_raster) * res[0]
rivers_distance

In [ ]:
# Plot rivers distance.
show(rivers_distance, transform=transform)
plt.show()

## Reclassify Distance Raster

Create a reclassified river distance raster based on the following distance values. Remember that the distance values are in meters!

| Min Distance (Inclusive) | Max Distance (Exclusive) |   Risk Level   | Risk Score |
| :----------------------: | :----------------------: | :------------: | :--------: |
|            ...           |      1 mile (1609 m)     | Very high risk |      5     |
|      1 mile (1609 m)     |     5 miles (8046 m)     |    High risk   |      4     |
|     5 miles (8046 m)     |    25 miles (40233 m)    |   Medium risk  |      3     |
|    25 miles (40233 m)    |    75 miles (120701 m)   |    Low risk    |      2     |
|    75 miles (120701 m)   |            ...           |  Very low risk |      1     |


Once you have created the reclassified distance raster, visualize it using the appropriate `transform` and the `"RdYlBu_r"` (reversed red-yellow-blue) colormap.

In [ ]:
rivers_reclass = np.full(shape, fill_value=np.NaN)
rivers_reclass[rivers_distance < 1609] = 5
rivers_reclass[(rivers_distance >= 1609) & (rivers_distance < 8046)] = 4
rivers_reclass[(rivers_distance >= 8046) & (rivers_distance < 40233)] = 3
rivers_reclass[(rivers_distance >= 40233) & (rivers_distance < 120701)] = 2
rivers_reclass[rivers_distance >= 120701] = 1
rivers_reclass

In [ ]:
show(rivers_reclass, transform=transform, cmap='RdYlBu_r')
plt.show()

## Calculate Weighted and Unweighted Risk Rasters

Create and visualize both a weighted and an unweighted risk raster. For the unweighed risk raster, just add up all the reclassified rasters. For the weighted risk raster, assign the following weights to each component.

| Component          | Weight |
| ------------------ | :----: |
| Distance to rivers |   35%  |
| Land cover         |   25%  |
| Slope              |   25%  |
| Elevation          |   15%  |

When visualizing the rasters, use the appropriate `transform` and the `"RdYlBu_r"` (reversed red-yellow-blue) colormap as before.

In [ ]:
unweighted_risk = rivers_reclass + land_reclass + slope_reclass + elev_reclass
unweighted_risk

In [ ]:
show(unweighted_risk, transform=transform, cmap='RdYlBu_r')
plt.show()

In [ ]:
weighted_risk = 0.35*rivers_reclass + 0.25*land_reclass + 0.25*slope_reclass + 0.15*elev_reclass
weighted_risk

In [ ]:
show(weighted_risk, transform=transform, cmap='RdYlBu_r')
plt.show()

## Mask the Risk Rasters

Because the rivers shapefile was confined to the boundaries of Ethiopia and the elevation DEM contained some missing data outside of the boundaries of Ethiopia, our risk rasters only have accurate values within the boundaries of Ethiopia. Hence we should mask the risk rasters to the boundaries of Ethiopia by marking all cells outside of Ethiopia as `np.NaN` to denote no data. This can be done as follows:

1. Use [`gpd.read_file()`](https://geopandas.org/docs/reference/api/geopandas.read_file.html#geopandas.read_file) to read in the boundaries of Ethiopia.
2. Use the geopandas [`.to_crs()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html) method to reproject the vector data into the coordinate system used for the raster data.
2. Use [`features.rasterize()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html#rasterio.features.rasterize) from RasterIO to convert the Ethiopia shapefile into a binary raster.
3. Use [boolean indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html#boolean-array-indexing) to replace all cells in the risk rasters that are not within Ethiopia with `np.NaN`.

Once you have masked the risk rasters, visualize them again using the appropriate `transform` and the `"RdYlBu_r"` (reversed red-yellow-blue) colormap.

In [ ]:
boundaries = gpd.read_file('./data/ethiopia.geojson').to_crs(crs)
boundaries

In [ ]:
boundaries.plot()
plt.show()

In [ ]:
# OPTION 1: convert to binary raster and reclassify
boundaries_raster = features.rasterize(boundaries.geometry, out_shape=shape, 
                                       fill=0, transform=transform,
                                       default_value=1)
boundaries_raster

In [ ]:
show(boundaries_raster, transform=transform)
plt.show()

In [ ]:
unweighted_risk_masked = np.where(boundaries_raster == 0, np.NaN, unweighted_risk)
unweighted_risk_masked

In [ ]:
show(unweighted_risk_masked, transform=transform, cmap='RdYlBu_r')
plt.show()

In [ ]:
weighted_risk_masked = np.where(boundaries_raster == 0, np.NaN, weighted_risk)
weighted_risk_masked

In [ ]:
show(weighted_risk_masked, transform=transform, cmap='RdYlBu_r')
plt.show()

In [ ]:
# OPTION 2: create mask and multiply
mask = features.rasterize(boundaries.geometry, out_shape=shape, fill=np.NaN,
                          transform=transform, default_value=1)
mask

In [ ]:
show(mask, transform=transform)
plt.show()

In [ ]:
unweighted_risk_masked = unweighted_risk * mask
unweighted_risk_masked

In [ ]:
show(unweighted_risk_masked, transform=transform, cmap='RdYlBu_r')
plt.show()

In [ ]:
weighted_risk_masked = weighted_risk * mask
weighted_risk_masked

In [ ]:
show(weighted_risk_masked, transform=transform, cmap='RdYlBu_r')
plt.show()

## Calculate Zonal Statistics

Determine the administrative zone with the highest risk as follows:

1. Use [`gpd.read_file()`](https://geopandas.org/docs/reference/api/geopandas.read_file.html#geopandas.read_file) to read in the boundaries of the administrative zones.
2. Use the geopandas [`.to_crs()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html) method to reproject the vector data into the coordinate system used for the raster data.
2. Visualize the administrative zone boundaries on top of the masked weighted risk raster.
3. Use the [`rasterstats`](https://pythonhosted.org/rasterstats/index.html) package to calculate the average weighted risk for each administrative zone.
4. Add the calculated average weighted risk values to the administrative zones GeoDataFrame.
5. Sort the administrative zones GeoDataFrame on the average weighted risk values and report the zone with the highest risk.
5. Visualize the administrative zones on a red-yellow-blue choropleth map where higher risk zones are colored red and lower risk zones are colored blue.

In [ ]:
zones = gpd.read_file('./data/zones.geojson').to_crs(crs)
zones.head()

In [ ]:
ax = zones.plot(facecolor='none', edgecolor='black')
show(weighted_risk_masked, ax=ax, cmap ='RdYlBu_r', transform=transform)
plt.show()

In [ ]:
stats = pd.DataFrame(zonal_stats(zones, weighted_risk_masked, affine=transform,
                                 stats='mean', nodata=np.NaN)) 
stats.head()

In [ ]:
zones_risk = zones.join(stats).rename(columns={'mean': 'mean_risk_score'})
zones_risk.head()

In [ ]:
zones_risk.sort_values(by='mean_risk_score', ascending=False).head()

In [ ]:
zones_risk.plot(column='mean_risk_score', cmap='RdYlBu_r', legend=True)
plt.show()

## Bonus: Create a Beautiful Raster Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.axis((bounds.left, bounds.right, bounds.bottom, bounds.top))
cx.add_basemap(ax=ax, crs=crs.to_string())
image = ax.imshow(weighted_risk_masked, cmap='RdYlBu_r',
                  extent=(bounds.left, bounds.right, bounds.bottom, bounds.top))
fig.colorbar(image)
plt.show()